In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login

login(token = "Your_huggingface_token")

# PostgreSQL connection parameters
db_params = {
    "dbname": "data",
    "user": "adityakumarraj",
    "password": "Aa%%407909013706",
    "host": "localhost",
    "port": "5432"
}

/Users/adityakumarraj/Desktop/sqlcoder-8b/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Create database connection
try:
    conn = psycopg2.connect(**db_params)
    print("Successfully connected to the database!")
    
    # Create SQLAlchemy engine
    engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}')
    
except Exception as e:
    print(f"Error connecting to the database: {e}")

Successfully connected to the database!


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


model_name = "defog/llama-3-sqlcoder-8b"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/Users/adityakumarraj/Desktop/sqlcoder-8b/venv/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/Users/adityakumarraj/Desktop/sqlcoder-8b/venv/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [4]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

db_params = {
    "dbname": "data",
    "user": "adityakumarraj",
    "password": "Aa%%407909013706",
    "host": "localhost",
    "port": "5432"
}
conn = psycopg2.connect(**db_params)
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}')

def get_schema_info():
    schema_query = """
    SELECT 
        table_name,
        column_name,
        data_type,
        is_nullable
    FROM information_schema.columns
    WHERE table_schema = 'public'
    ORDER BY table_name, ordinal_position;
    """
    return pd.read_sql_query(schema_query, engine)

# Get schema information
schema_df = get_schema_info()
print("\nDatabase Schema:")
display(schema_df)


Database Schema:


,table_name,column_name,data_type,is_nullable
0,costs_data,id,bigint,NO
1,costs_data,account_id,bigint,YES
2,costs_data,product_name,text,YES
3,costs_data,date_day,date,YES
4,costs_data,category,text,YES
...,...,...,...,...
112,projectdata,gpuquota,integer,YES
113,projectdata,cpuquota,integer,YES
114,projectdata,memoryquota,integer,YES
115,projectdata,createdat,timestamp without time zone,YES


In [5]:
def get_nodedata_schema():
    schema_query = """
    SELECT 
        column_name,
        data_type,
        is_nullable
    FROM information_schema.columns
    WHERE table_schema = 'public' 
    AND table_name = 'nodedata'
    ORDER BY ordinal_position;
    """
    return pd.read_sql_query(schema_query, engine)

# Get schema information
nodedata_schema = get_nodedata_schema()
print("\nNodedata Table Schema:")
display(nodedata_schema)


Nodedata Table Schema:


,column_name,data_type,is_nullable
0,nodeid,text,NO
1,nodename,text,YES
2,clustername,text,YES
3,instancetype,text,YES
4,tags,text,YES
5,created,timestamp without time zone,YES
6,snapshottime,timestamp without time zone,YES
7,platform,text,YES


In [ ]:
# --- Initialization section ---
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import sqlparse
import psutil

available_memory = psutil.virtual_memory().available

model_name = "defog/llama-3-sqlcoder-8b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set prompt template (initialized once)
prompt = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Generate a SQL query to answer this question: `{user_question}`

### Database Schema
CREATE TABLE nodedata (
nodeid TEXT PRIMARY KEY, -- Unique identifier for the node
nodename TEXT, -- Name of the node
clustername TEXT, -- Name of the Kubernetes or compute cluster
instancetype TEXT, -- EC2 instance type or VM size (e.g., c5.18xlarge, p4d.24xlarge)
tags TEXT, -- JSON-style string of metadata tags (key/value pairs)
created TIMESTAMP WITHOUT TIME ZONE, -- When the node was first created
snapshottime TIMESTAMP WITHOUT TIME ZONE, -- When this snapshot of node metadata was taken
platform TEXT -- Runtime platform or orchestration tool (e.g., runai, kubernetes)
);

If you cannot answer the question with the available database schema, return 'I do not know'
The following SQL query best answers the question `{user_question}`:
```sql"""

if available_memory > 15e9:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        device_map="auto",
        use_cache=True,
    )


# --- Generation function ---
def generate_query(question: str) -> str:
    formatted_prompt = prompt.format(user_question=question)
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=400,
            do_sample=False,
        )

    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    raw_sql = outputs[0].split("```sql")[-1].split("```")[0].strip()
    return sqlparse.format(raw_sql, reindent=True)  


def query_database(question: str, engine):
    print(f"Question: {question}\n")
    sql_query = generate_query(question)
    print("Generated SQL Query:")
    print(sql_query)
    print("\nQuery Result:")

    try:
        result = pd.read_sql_query(sql_query, engine)
        display(result)
    except Exception as e:
        print(f"Error executing query: {e}")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/Users/adityakumarraj/Desktop/sqlcoder-8b/venv/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/Users/adityakumarraj/Desktop/sqlcoder-8b/venv/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Some parameters are on the meta device because they were offloaded to the disk.


In [7]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import pandas as pd


# # Pre-build static schema string
# schema_str = "\n".join(
#     f"Column {row['column_name']} with type {row['data_type']}"
#     for _, row in nodedata_schema.iterrows()
# )

# # Optional cache to avoid regenerating for repeated questions
# _sql_cache = {}

# def generate_sql_for_nodedata(question: str) -> str:
#     # Return cached result if available
#     if question in _sql_cache:
#         return _sql_cache[question]

#     # Assemble prompt
#     prompt = f"""
# ### Task
# Generate a PostgreSQL query to answer the following question about the nodedata table:
# {question}

# ### Database Schema
# CREATE TABLE nodedata (
# nodeid TEXT PRIMARY KEY, -- Unique identifier for the node
# nodename TEXT, -- Name of the node
# clustername TEXT, -- Name of the Kubernetes or compute cluster
# instancetype TEXT, -- EC2 instance type or VM size (e.g., c5.18xlarge, p4d.24xlarge)
# tags TEXT, -- JSON-style string of metadata tags (key/value pairs)
# created TIMESTAMP WITHOUT TIME ZONE, -- When the node was first created
# snapshottime TIMESTAMP WITHOUT TIME ZONE, -- When this snapshot of node metadata was taken
# platform TEXT -- Runtime platform or orchestration tool (e.g., runai, kubernetes)
# );

# ### Response
# Here's a PostgreSQL query to answer your question:
# """
#     # Tokenize and generate within no_grad to speed up
#     inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
#     with torch.no_grad():
#         outputs = model.generate(
#             **inputs,
#             max_new_tokens=150,      # adjust as needed
#             do_sample=False,         # greedy decoding for speed
#             pad_token_id=tokenizer.eos_token_id
#         )

#     # Decode and extract SQL part
#     text = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     sql = text.split("Here's a PostgreSQL query to answer your question:")[-1].strip()

#     # Cache and return
#     _sql_cache[question] = sql
#     return sql


# def query_nodedata(question: str):
#     print(f"Question: {question}\n")
#     sql_query = generate_sql_for_nodedata(question)
#     print("Generated SQL Query:")
#     print(sql_query)
#     print("\nQuery Result:")

#     try:
#         result = pd.read_sql_query(sql_query, engine)
#         display(result)
#     except Exception as e:
#         print(f"Error executing query: {e}")

In [9]:
question = "Show me the first 5 rows from the nodedata table"
generate_query(question)

/Users/adityakumarraj/Desktop/sqlcoder-8b/venv/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'SELECT *\nFROM nodedata\nORDER BY created ASC\nLIMIT 5;'

In [11]:

question =   "Show me the nodes created in the last 7 days",
generate_query(question) 
    # Multiple conditions


"SELECT n.nodename\nFROM nodedata n\nWHERE n.created > CURRENT_DATE - INTERVAL '7 days'\nORDER BY n.created DESC NULLS LAST;"

In [ ]:
question ="What is the distribution of instance types?",
generate_query(question)  
    # Complex combinations


'SELECT instancetype,\n       COUNT(*) AS COUNT\nFROM nodedata\nGROUP BY instancetype\nORDER BY COUNT DESC NULLS LAST;'

In [ ]:
question ="What are all the nodes in the 'sensei-eks01-prod-cluster' cluster?",
generate_query(question) 
    # Pattern matching


"SELECT nd.nodename\nFROM nodedata nd\nWHERE nd.clustername ='sensei-eks01-prod-cluster';"

In [ ]:
question ="Show the number of nodes created per day"
generate_query(question) 

"SELECT date_trunc('day', n.created) AS creation_date,\n       COUNT(*) AS num_nodes\nFROM nodedata n\nGROUP BY creation_date\nORDER BY creation_date NULLS LAST;"

In [16]:
question =    "How many nodes are there per cluster?",
generate_query(question) 

'SELECT n.clustername,\n       COUNT(n.nodeid) AS num_nodes\nFROM nodedata n\nGROUP BY n.clustername\nORDER BY num_nodes DESC NULLS LAST;'

In [17]:
question =  "What are the instance types used in each platform?",
generate_query(question) 
    # Aggregations

'SELECT n.platform,\n       n.instancetype\nFROM nodedata n\nGROUP BY n.platform,\n         n.instancetype\nORDER BY n.platform,\n         n.instancetype NULLS LAST;'

In [19]:
question ="Show me the clusters that have more than 5 nodes, grouped by platform",
generate_query(question) 
    # Specific lookups

'SELECT n.clustername,\n       COUNT(n.nodeid) AS node_count,\n       n.platform\nFROM nodedata n\nGROUP BY n.clustername,\n         n.platform\nHAVING COUNT(n.nodeid) > 5\nORDER BY n.clustername,\n         n.platform NULLS LAST;'

In [18]:
question ="Find all nodes that have 'compute' in their name",
generate_query(question) 


"SELECT n.nodename\nFROM nodedata n\nWHERE n.nodename ilike '%compute%';"